# How to Examine a Stored Dataset's Chunk Shape

The objective of this notebook is to learn how to examine a stored dataset and understand if it is chunked and, if so, what its "[**chunk shape**](../back/Glossary.md#term-Chunk-shape)" is.
To do this, we will utilize an existing dataset from the HyTEST OSN, take a guided tour of the data, and show how to figure out its chunk shape.

In [ ]:
import xarray as xr
import fsspec

## Accessing the Dataset

Before we can open the dataset, we must first get a mapper that will easily allow for [xarray](https://docs.xarray.dev/en/stable/index.html) to open the dataset.
To do this, we will use [fsspec](https://filesystem-spec.readthedocs.io/en/latest/) to perform an anonymous read from an endpoints outside of S3, using the S3 API (i.e., the HyTEST OSN).
This requires us to set up an S3 file system and feed it the endpoint URL.
We can then point the the file system to our dataset (in this case the PRISM V2 Zarr store) and get a mapper to the file.

In [ ]:
fs = fsspec.filesystem(
    's3',
    anon=True,   # anonymous = does not require credentials
    client_kwargs={'endpoint_url': 'https://usgs.osn.mghpcc.org/'}
)
file = fs.get_mapper('s3://mdmf/gdp/PRISM_v2.zarr/')

Now that we have our file mapper, we can open the dataset using [`xarray.open_dataset()`](https://docs.xarray.dev/en/stable/generated/xarray.open_dataset.html) with `zarr` specified as our engine.

```{note}
The xarray loader is "lazy", meaning it will read just enough of the data to make decisions about its shape, structure, etc.
It will pretend like the whole dataset is in memory (and we can treat it that way), but it will only load data as required.
```

In [ ]:
ds = xr.open_dataset(file, engine='zarr')
ds

The "rich" HTML output to show the [`xarray.Dataset`](https://docs.xarray.dev/en/stable/generated/xarray.Dataset.html) includes a lot of information, some of which is hidden behind toggles.
Click on the icons to the right to expand and see all the metadata available for the dataset.
The page icon will display attributes attached to the data, while the database icon will display information about the dataset.

Notable observations: 
- **Dimensions**: This dataset is 3D, with data being indexed by `lat`, `lon`, and `time` (setting  aside `tbnd` for the moment; it is a special case).
  Looking at the "Dimensions" line, you can see that each of these dimensions is quantified (i.e., how many unique values are available in each dimension).
    - **lat** = 621
    - **lon** = 1405
    - **time** = 1555
- **Coordinates**: These are the convenient handles by which dimensions can be referenced.
  In this dataset, a coordinate can be used to pick out a particular cell of the array.
  Asking for cells where say `lat=49.9` is possible because these coordinates map the meaningful values of latitude to the behind-the-scenes cell index needed to fetch the value. 
- **Data Variables**: The variables are `tmx`, `ppt`, and `tmn`, which are associated with three coordinates by which data values are located in space and time (the "Dimensions"). 
- **Indexes**: This is an internal data structure to help xarray quickly find items in the array.
- **Attributes**: Arbitrary metadata that has been given to the dataset. 

Let's look at one of the data variables to learn more about it. 

### Variable = [`xarray.DataArray`](https://docs.xarray.dev/en/stable/generated/xarray.DataArray.html#xarray.DataArray)

Each data variable is its own N-dimensional array (in this case, 3-dimensional, indexed by `lat`, `lon`, and `time`).
We can look at the individual variables by examining its array separately from the dataset: 

In [ ]:
ds.tmn

Note from the top line that this variable is indexed as a tuple in `(time, lat, lon)`.
So, behind the scenes, there is an array whose first index (for time) is a value between 0 and 1555. 
But how do we know the time value of index 0 (or any index, really)?
The "Coordinates" are the lookup table to say what "real" time value is associated with each index address. 

You'll notice that the data description in this case is merely "1356745275 values with dtype=float64" with no indication as to how it is chunked.
Assuming our 3-D array is fully populated, this value makes sense:

In [ ]:
# time  lat  lon
1555 * 621 * 1405

In terms of chunking, this is where it gets interesting.
If you thoroughly looked at the rich HTML output, you may have noticed that there is no reference to chunking anywhere.
Occasionally, some datasets will include some info in their "Attributes" that potentially indicate how the data is chunked.
For example, some may include a `_ChunkSizes` that gives the chunk shape of the data, expressed as a tuple to match the dimensions.
However, we should be skeptical of any information on how the data is chunked in the "Attributes".
Virtually anything can be set as an attribute on the dataset, and it does not affect the internal structure of the dataset **AT ALL**.
Therefore, we need to directly access the data in a way that returns the true chunk shape of the stored dataset.

To do this, we can simply check the variables "encoding".
This returns metadata that was used by xarray when reading the data.

In [ ]:
ds.tmn.encoding

From here we can see two keys dealing with chunks: `'chunks'` and `'preferred_chunks'` - both contain the same information of how the data is chunked.
The difference between the two is `'chunks'` is the actual chunk shape of the chunks stored on disk (what is commonly termed "[**stored chunks**](../back/Glossary.md#term-Stored-chunks)"), while `'preferred_chunks'` is the chunk shape that the engine chose to open the dataset with.
Generally, these are the same, but they may be different if the engine you use has not been set to equate them.
Therefore, our data has a stored chunk shape of `{'time': 68, 'lat': 131, 'lon': 294}`.

## Getting the Chunking When Reading Data

While checking the "encoding" of the variable can tell you what the dataset's stored chunk shape is, it is typically easier to do this in one step when you open the dataset.
To do this, all we need is to add a another keyword when we open the dataset with xarray: `chunks={}`.
As per the [`xarray.open_dataset` documentation](https://docs.xarray.dev/en/stable/generated/xarray.open_dataset.html): 

> `chunks={}` loads the data with dask using the engine’s preferred chunk size, generally identical to the format’s chunk size.

In other words, using `chunks={}` will load the data with chunk shape equal to `'preferred_chunks'`.
Let's check this out and see how our data looks when we include this keyword when opening.

In [ ]:
ds = xr.open_dataset(file, engine='zarr', chunks={})
ds

Now if we inspect the data variables metadata, we will see that the data is now read in as a dask array.
Let's look at at the `tmn` variable again to simplify this.

In [ ]:
ds.tmn

As we can see the data is chunked into chunks of shape `(68, 131, 294)` with a "[**chunk size**](../back/Glossary.md#term-Chunk-size) of ~20 MiB.
This is exactly what we saw when looking at the encoding.
So, this additional keyword worked as expected and gives us a standard way to open chunked datasets using the stored chunk shape as our chunk shape!

Note that the coordinate variables themselves (`lat`, `lon`, and `time`) are stored as single unchunked arrays of data.
Recall that these are used to translate a coordinate value into the index of the corresponding array.
Therefore, these coordinate arrays will always be needed in their entirity.
So, they are included in each chunk such that they read whenever a chunk is read, and they do not affect how the data representing the data variables is chunked.  


## Changing the Chunk Shape and Size

Now that we know our stored chunk shape and size or how to find them, they may not always be the optimal choice for performing analysis.
For example, [Zarr recommends a stored chunk size of at least 1 MB uncompressed](https://zarr.readthedocs.io/en/stable/tutorial.html#chunk-size-and-shape) as they give better performance.
However, [dask recommends chunk sizes between 10 MB and 1 GB for computations](https://docs.dask.org/en/stable/array-chunks.html#specifying-chunk-shapes), depending on the availability of RAM and the duration of computations.
Therefore, our stored chunk size may not be large enough for optimal computations.
Thankfully, stored chunks do not need to be the same size as those we use for our computations.
In other words, we can group multiple smaller stored chunks together when performing our computations.
Xarray makes this easy by allowing us to adjust the chunk shape and size, either as we load the data or after.

Let's show how this works by increasing our chunks of the minimum monthly temperature to a size of ~500 MiB.
To do so when reading in the data, all we need to do is actually specify the chunk shape to `chunks`.
For our example, let's do chunks of shape: `{'time': 150, 'lat': 310, 'lon': 1405}`.

In [ ]:
# Note we drop the other variables and select tmn when reading the data
ds_tmn = xr.open_dataset(file, engine='zarr',
                         chunks={'time': 150, 'lat': 310, 'lon': 1405},
                         drop_variables=['ppt', 'time_bnds', 'tmx', 'crs']).tmn
ds_tmn

Nice!
As we can see, the chunk shape is now reflected in data description with about the right chunk size we wanted.
However, we did get a warning indicating that:

```
UserWarning: The specified chunks separate the stored chunks along dimension X starting at index i. This could degrade performance. Instead, consider rechunking after loading.
```

```{important}
This warning is telling us that the chunk shape we have chosen is not a multiple (or grouping) of the the stored chunks, and if we really want this chunk shape, we should rechunk.
```

Oops, as we are not attached to this chunk shape nor wanting to rechunk the data (see the [Why (re)Chunk Data? notebook](WhyChunk.ipynb) for reasons why you might), we need to select a chunk shape that is a **multiple** of the stored chunks.
This time, let's try: `{'time': 68*3, 'lat': 131*3, 'lon': 294*3}`.
This should increase our original chunk size (~20 MiB) by a factor of 27 ($3^3 = 27$) - close to the ~500 MiB we are wanting.

In [ ]:
ds_tmn = xr.open_dataset(file, engine='zarr',
                         chunks={'time': 68 * 3, 'lat': 131 * 3, 'lon': 294 * 3},
                         drop_variables=['ppt', 'time_bnds', 'tmx', 'crs']).tmn
ds_tmn

Look at that, no warning and close to the chunk size we wanted!

As a final note, we selected our chunk shape while reading in the data.
However, we could change them after using [`xarray.Dataset.chunk()`](https://docs.xarray.dev/en/stable/generated/xarray.Dataset.chunk.html).

In [ ]:
ds.tmn.chunk({'time': 68 * 4, 'lat': 131 * 4, 'lon': 294 * 4})

`````{admonition} Not Recommended
Warning: We do not recommend using this method as you will not get the same warning notifying you that the chosen chunk shape does not match a multiple of the stored chunk shape. Therefore, if you choose a non-multiple chunk shape you could slow down your whole workflow as the data will have to be rechunked to meet your requested chunk shape.
`````